In [ ]:
#import packages
import numpy as np
import pandas as pd
from numpy import array
#Creating the stacked LSTM Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
#to plot within notebook
import matplotlib.pyplot as plt
import tensorflow as tf
# Calculating RMSE Performance
import math
from sklearn.metrics import mean_squared_error
%matplotlib inline
#setting figure size
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 20,10
#for normalizing data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))

In [ ]:
# Converting the array of values into a dataset matrix
def create_dataset(dataset, time_step=1):
    dataX, dataY= [],[]
    for i in range(len(dataset)-time_step-1):
        a= dataset[i:(i+time_step), 0]
        dataX.append(a)
        dataY.append(dataset[i+time_step, 0])
    return np.array(dataX), np.array(dataY)

In [ ]:
def predict(data):
    df1 = data.reset_index()['Close']
    df1= scaler.fit_transform(np.array(df1).reshape(-1, 1))
    # SPLITTING INTO TRAIN AND TEST
    #deciding the split size
    trainSize = int(len(df1)*0.65)
    testSize= len(df1)- trainSize

    #Making the splits
    trainData, testData = df1[0:trainSize, :],df1[trainSize:len(df1), :1 ] 
    time_step= 320
    X_train, Y_train= create_dataset(trainData, time_step)
    X_test, Y_test= create_dataset(testData, time_step)
    #Before going into LSTM, we have to make our data 3D
    X_train= X_train.reshape(X_train.shape[0], X_train.shape[1],1)
    X_test= X_test.reshape(X_test.shape[0], X_test.shape[1],1)
    
    #making the model
    model=Sequential()
    model.add(LSTM(50,return_sequences=True,input_shape=(320,1)))
    model.add(LSTM(50,return_sequences=True))
    model.add(LSTM(50))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error',optimizer='adam')
    
    #Training the model
    model.fit(X_train, Y_train, validation_data= (X_test, Y_test), epochs= 15, batch_size=128, verbose=1)
    #Doing the predictions
    train_predict= model.predict(X_train)
    test_predict= model.predict(X_test)
    # transforming back to original , to perform RMSE
    train_predict= scaler.inverse_transform(train_predict)
    test_predict= scaler.inverse_transform(test_predict)
    
    #Plotting the training and testing data
    look_back= 320 #time stamp
    #Training data
    trainPredictPlot= np.empty_like(df1)
    trainPredictPlot[:, :]= np.nan
    trainPredictPlot[look_back: len(train_predict)+look_back, :]= train_predict

    #Test Data
    testPredictPlot= np.empty_like(df1)
    testPredictPlot[:, :]= np.nan
    testPredictPlot[len(train_predict)+(look_back*2) +1: len(df1) -1 , :]= test_predict

    #Plotting the baseline and predictions

    plt.plot(scaler.inverse_transform(df1))
    plt.plot(trainPredictPlot)
    plt.plot(testPredictPlot)
    plt.show()
    
    x_input=testData[(len(testData)-320) :].reshape(1,-1)
    temp_input=list(x_input)
    temp_input=temp_input[0].tolist()
    
    
    #MAKING PREDICTIONS FOR THE NEXT 15 DAYS
    
    lst_output=[]
    n_steps=320
    i=0
    while(i<15):

        if(len(temp_input)>320):
            #print(temp_input)
            x_input=np.array(temp_input[1:])
            x_input=x_input.reshape(1,-1)
            x_input = x_input.reshape((1, n_steps, 1))
            #print(x_input)
            yhat = model.predict(x_input, verbose=0)
            temp_input.extend(yhat[0].tolist())
            temp_input=temp_input[1:]
            #print(temp_input)
            lst_output.extend(yhat.tolist())
            i=i+1
        else:
            x_input = x_input.reshape((1, n_steps,1))
            yhat = model.predict(x_input, verbose=0)
            print(yhat[0])
            temp_input.extend(yhat[0].tolist())
            print(len(temp_input))
            lst_output.extend(yhat.tolist())
            i=i+1

    day_new=np.arange(1,321)
    day_pred=np.arange(321,336)

    plt.plot(day_new,scaler.inverse_transform(df1[(len(df1)-320):]))
    plt.plot(day_pred,scaler.inverse_transform(lst_output))
    plt.show()     
    # Combining the data into one plot , checking only the predicted plot
    df3=df1.tolist()
    df3.extend(lst_output)
    df3=scaler.inverse_transform(df3[len(df1):]).tolist()
    
    plt.plot(df3)
    plt.show()
    
    diff = []
    for i in range(len(df3)-1):
        diff.append(df3[i][0] - df3[i+1][0])

    avg = sum(diff)/len(diff)
    
    return avg

In [ ]:
banks = {
    "AXIS_BANK": "not defined",
    "BAJAJ_FINANCE" : "not defined",
    "BAJAJ_FINSERV" : "not defined",
    "HDFC_BANK" : "not defined",
    "ICICIBANK" : "not defined",
    "INDUSIND_BANK" : "not defined",
    "KOTAK_MAHINDRA" : "not defined",
    "SBI" : "not defined"  
}

for i in banks:
    
    path = "./Dataset/BanksDataset/"+i+".csv"
    data = pd.read_csv(path)
    if banks[i] == "not defined":
        
        x = predict(data)
        banks[i] = x
        if banks[i] < 0:
            print("You can Invest in "+i+"!")
        elif banks[i] > 0:
            print("You should not invest in "+i+"!")
        else:
            print("You may Invest in "+i+"!")